대부분의 프로그램은 정해진 범위에서만 돌아가며 한번에 한줄만 실행됩니다.

근데 조금더 알아보면, 프로그램은 사실 동시에 여러 공간에서 동작하는게 가능합니다.

이를 동시성이라 부르며, 동시성은 다음의 이점이 있습니다.

- (Performance) 동시성으로 인해 작업 처리시간이 짧아집니다.

- (Robustness) 하나의 시스템이 망가져도 다른시스템이 온전하게 작업을 처리할수 있습니다.

- (Simplicity) 복잡한 작업을 잘개 쪼개 쉬운 작업으로 만들 수 있습니다.

- (Communication) 다른사람과 즐겁게 공유 가능합니다.

## Concurrency

보통 컴퓨터에서 뭔가를 기다리는 경우는 다음 두 경우중 하나입니다.

- I/O bound

> Input, Output 작업에 대부분을 보내는 프로세스 상태입니다.

> 일반적으로 CPU가 메모리보다 훨씬 빠르기 때문에 대기상태는 거의 다 I/O bound입니다.

- CPU bound

> CPU연산에 대부분을 보내는 프로세스 상태입니다.

> 보통 CPU에서 빡센 과학적, 수학적, 그래픽적 계산을 할 때 이 상태가 됩니다.

동시성과 관련해서 다음 두 종류의 용어를 알아 둘 필요가 있습니다.

- 동기 (synchronous)

> 하나의 작업 이후 다음 작업이 오는 경우

- 비동기 (asynchronous)

> 여러 작업들이 순서 상관 없이 독립적으로 이루어 지는 경우

웹사이트를 예로 들어볼까요? 

접속하는 사용자마다 페이지를 렌더링 해주는 작업은 간단하고 빠른 작업입니다.

접속자가 많아지더라도 이정도 시간은 기다릴 수 있습니다.

**하지만 파일 업로드와 같은 긴 시간이 걸리는 작업은 어떨까요?**

동기 구조의 웹서버는 이 작업을 처리할 수가 없습니다. 

누군가 파일을 업로드하는 동안 서버는 다른사람들에게 페이지를 렌더링해 줄수 없습니다.

**이때 우리는 비동기로 한개의 서버에서 동시에 여러작업을 처리할 수 있어야 합니다.**

## Queues

Queues는 한국말로 대기열이라고 불리며 간단히 말해 선입선출입니다.

설거지를 하는 사람을 예로 들어봅니다.

1. 혼자 일하기 (대기열 1개)

> 접시들을 들어온 순서대로 씻은뒤 말리는 일을 반복합니다.

2. 둘이 일하기 (대기열 2개)

> 한명은 접시들을 들어온 순서대로 씻고 다른사람에게 건내줍니다.

> 다른 한명은 접시를 받는 순서대로 대로 말립니다.

상식적으로 혼자보다는 둘이 하는게 더 빠르지만, 이경우 병목현상이 생길 수 있습니다.

둘중 한명이 작업속도가 월등히 빠를때, 전체 작업 속도는 둘중 가장 느린 사람의 속도와 같아집니다.

이때 동기 방식의 프로세스를 묘사하자면 다음과 같습니다.

> 접시를 씻는 사람은 

> 접시를 말리는 사람이 접시를 다 말릴 때까지 기다렸다가 접시를 건내줍니다.

비동기 방식의 프로세스는 다음과 같습니다.

> 접시를 씻는 사람은 접시를 다 씻고 둘 사이에 접시를 쌓아놓습니다.

## Processes

프로세스는 컴퓨터의 기본 작업 처리 단위입니다.

파이썬의 multiprcessing이란 모듈에서 Queue를 사용해 볼 수 있습니다.

앞서 설거지의 예시를 파이썬 코드로 작성해 봅시다.

접시를 씻는사람 1명, 말리는사람 여러명의 경우를 예로 들어봅시다.

In [8]:
import multiprocessing as mp
import time

In [9]:
def washer(dishes, output):
    for dish in dishes:
        print('Washing', dish, 'dish')
        print("씻는중")
        time.sleep(2)
        output.put(dish)
                
def dryer(input):
    while True:
        dish = input.get()
        print('Drying', dish, 'dish')
        input.task_done()

In [12]:
dish_queue = mp.JoinableQueue()
dryer_proc = mp.Process(target=dryer, args=(dish_queue,))
dishes = ['salad', 'bread', 'entree', 'dessert']

- DAEMON

> 데몬은 사용자의 서비스 요청에 대해 대기하고 있는 서비스 처리자 입니다.

In [13]:
dryer_proc.daemon = True
dryer_proc.start()
washer(dishes, dish_queue)
dish_queue.join()

Washing salad dish
씻는중
Drying salad dish
Washing bread dish
씻는중
Drying bread dish
Washing entree dish
씻는중
Drying entree dish
Washing dessert dish
씻는중
Drying dessert dish


## Threads

프로세스대신 스레드를 사용해 볼수도 있습니다.

스레드는 한개의 프로세스 안에서 실행되며 프로세스 모든 부분에 접근 가능합니다. 

비유하자면, 프로세스가 작업자 한명이라면 스레드는 한사람 안의 여러 인격입니다.

설거지 예제코드를 스레드로 만들어 보겠습니다.

In [16]:
import threading, queue, time

In [17]:
def washer(dishes, dish_queue):
    for dish in dishes:
        print("Washing", dish)
        time.sleep(2)
        dish_queue.put(dish)

def dryer(dish_queue):
    while True:
        dish = dish_queue.get()
        print("Drying", dish)
        time.sleep(4)
        dish_queue.task_done()

In [18]:
dish_queue = queue.Queue()
for n in range(2):
    dryer_thread = threading.Thread(target=dryer, args=(dish_queue,))
    dryer_thread.start()

In [19]:
dishes = ['salad', 'bread', 'entree', 'dessert']
washer(dishes, dish_queue)
dish_queue.join()

Washing salad
WashingDrying bread
 salad
WashingDrying bread entree

WashingDrying entree
 dessert
Drying dessert


> 출력 상태가 살짝 이상합니다. 다 이유가 있습니다. 뒤에서 설명합니다.

스레드에는 종료함수가 없고 종료를 알리지도 않습니다. 

스레드를 컨트롤 하는 것은 엄청나게 위험할 수 있습니다. 

C나 C++ 언어에서 스레드를 컨트롤하는 것은 디버깅하기 매우 어려운 버그를 유발합니다.

위의 예제코드에서는 다행히도 스레드들이 전역변수를 공유하지 않기 때문에 오류가 발생하지 않았습니다.

**귀신을 예로들어 멀티스레드의 위험성을 설명해 보겠습니다.**

> 당신은 유령의 집에 살고있습니다.

> 집안의 유령들은 서로의 상태를 모릅니다. 유령을 스레드로 비유할 수 있습니다.

> 유령들은 이 집의 가구들을 마음대로 옮기고 없에버릴 수 있습니다.

> 당신이 집의 촛불을 켜려고 하는 순간 유령이 촛대를 폭탄으로 바꿀 수도 있습니다.

집안의 가구들이 바로 프로그램의 변수입니다.

이처럼 멀티스레드는 각각의 스레드가 공통의 전역변수를 공유하지 않는다는 가정하에

안전하고 빠른 동시성을 보장할 수 있습니다.

안전한 멀티스레드를 위해 변수를 잠궈버릴수도 있습니다. 

귀신으로 예를 들자면 특정 가구에 퇴마를 수행해 한동안 귀신이 접근하지 못하도록 막는것과 비슷합니다.

다만, 이런방식은 어떤 변수를 잠궜다가 풀어줄지 고려하게 되면서 프로그램이 더 복잡해지도록 만듭니다.

사실상 파이썬에서는 GIL(Global Interpreter Lock)이라 불리는 변수마다의 안전장치가 있습니다.

따라서 CPU-bound 상태에서 스레드를 이용해 시간단축을 할 수 없습니다.

결론적으로 파이썬에서 다음과 같은 방침을 따르는게 좋습니다.

- 스레드는 I/O bound 문제를 위해 사용합니다.

> 파이썬에서 I/O bound 작업은 스레드를 이용해 동시처리가 가능합니다.

- CPU bound에 대해서는 프로세스와 네트워크, 이벤트등을 사용하도록 합니다.

## asyncio

[파이썬에서의 동시성, 병렬성](https://www.slideshare.net/deview/2d4python)

[파이썬에서의 동시성, 병렬성 2](https://okky.kr/article/388971)

## Redis

지금까지 설거지를 예로들어 프로세스와 스레드를 설명했습니다.

하지만 지금까지 한가지 가정한게 있습니다. 바로 한대의 머신에서 프로그래밍이 이루어진다는 것이죠.

이번엔 스케일을 확장해 여러대의 머신에서 실행해 보는건 어떨까요?

지금부터 네트워크를 활용한 프로그래밍을 해봅시다! 간단한 예시로 redis서버를 이용합니다.

In [1]:
! pip install redis
! yes | apt-get install redis-server

    100% |████████████████████████████████| 71kB 764kB/s ta 0:00:01
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libjemalloc1 redis-tools
Suggested packages:
  ruby-redis
The following NEW packages will be installed:
  libjemalloc1 redis-server redis-tools
0 upgraded, 3 newly installed, 0 to remove and 2 not upgraded.
Need to get 517 kB of archives.
After this operation, 1505 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu xenial/universe amd64 libjemalloc1 amd64 3.6.0-9ubuntu1 [78.9 kB]
Get:2 http://archive.ubuntu.com/ubuntu xenial/universe amd64 redis-tools amd64 2:3.0.6-1 [95.3 kB]
Get:3 http://archive.ubuntu.com/ubuntu xenial/universe amd64 redis-server amd64 2:3.0.6-1 [343 kB]
Fetched 517 kB in 5s (95.5 kB/s)       
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend 

**./washer.py**
```python
import redis, time
conn = redis.Redis()
print('Washer is starating')
dishes = ['salad', 'bread', 'entree', 'dessert']
for dish in dishes:
    time.sleep(1)
    msg = dish.encode('utf-8')
    conn.rpush('dishes', msg)
    print('Washed', dish)
conn.rpush('dishes', 'quit')
print('Washer is done')
```

**./dryer.py**
```python
import redis
conn = redis.Redis()
print('Dryer is starting')
while True:
    msg = conn.blpop('dishes')
    if not msg:
        break
    val = msg[1].decode('utf-8')
    if val == 'quit':
        break
    print('Dried', val)
print('Dishes are dried')
```

In [1]:
! /etc/init.d/redis-server restart
! python dryer.py & python washer.py

Stopping redis-server: redis-server.
Starting redis-server: redis-server.
Dryer is starting
Washer is starating
Washed salad
Dried salad
Washed bread
Dried bread
Washed entree
Dried entree
Washed dessert
Dried dessert
Washer is done
Dishes are dried


## Beyond Queues

## Networks

지금까지 동시성에 대해 논의할 때, Redis를 제외하고는 모두 네트워크를 사용하지 않았습니다.

이전에 말했듯 네트워크를 사용하면 한개 이상의 머신에서 작업을 수행하는게 가능해집니다.

네트워크를 이용해 작업을 처리할 때는 몇가지 패턴이 있는데 다음과 같습니다.

- request-reply (client-server)

> 대표적으로 웹서버가 있습니다. 클라이언트가 요청을 보내면 서버가 응답합니다.

> 서버가 응답하기 전까지 클라이언트는 기다려야 합니다. 동기식 처리방식입니다.

- push

> 중앙에서 작업을 처리할 수 있는 시스템으로 데이타를 전송합니다.

> 대표적으로 로드벨런서-다중웹서버 시스템이 이와 같습니다.

- pull

> 데이타를 받을수 있는 모든곳으로 부터 데이타를 받습니다.

> 서버에서 일어나는 일을 모두 기록하는 로그가 이와 같습니다.

- **publush-subscribe**

> 쉽게말해 텔레비전 방송처럼 동작하는 시스템입니다.

> 중앙에서 데이타를 뿌리면 받을수 있는 곳에서 받습니다.

> 데이타를 받는 위치에 있는 시스템은 어떤데이타만 받을지 선택 가능합니다.

## The Publish-Subscribe Model

Publish-Subscribe 모델은 Queue가 아니라 Broadcast라고 부릅니다.

Publish하는 시스템은 어떤 시스템이 Subscribe할지 알지 못합니다.

Redis서버가 간단하게 이 모델을 구축할 수 있습니다.

**./pub.py**
```python
import redis, random
conn = redis.Redis()
cats = ['siamese', 'persian', 'maine coon', 'norwegian forest']
hats = ['stovepipe', 'bowler', 'tam-o-shanter', 'fedora']
for msg in range(10):
    cat = random.choice(cats)
    hat = random.choice(hats)
    print('Publish: %s wears s %s' % (cat, hat))
    conn.publish(cat, hat)
```

**./sub.py**
```python
import redis
conn = redis.Redis()
topics = ['maine coon', 'persian']
sub = conn.pubsub()
sub.subscribe(topics)
for msg in sub.listen():
    if msg['type'] == 'message':
        cat = msg['channel']
        hat = msg['data']
        print('Subscribe: %s wears a %s' % (cat, hat))
```

In [4]:
! /etc/init.d/redis-server restart
! python sub.py & python pub.py

Stopping redis-server: redis-server.
Starting redis-server: redis-server.
Publish: persian wears s fedora
Publish: persian wears s tam-o-shanter
Publish: persian wears s tam-o-shanter
Publish: maine coon wears s tam-o-shanter
Publish: siamese wears s fedora
Publish: norwegian forest wears s stovepipe
Publish: maine coon wears s stovepipe
Publish: maine coon wears s fedora
Publish: persian wears s fedora
Subscribe: b'maine coon' wears a b'fedora'
Subscribe: b'persian' wears a b'fedora'
Publish: norwegian forest wears s fedora


## ZeroMQ

ZeroMQ 패키지는 Redis와 다르게 중앙서버가 없습니다.

복잡한 설치 없이도 예제코드를 돌려볼 수 있습니다. 어떤 경우엔 Redis보다 좋다고 합니다!

## TCP/IP

여기까지 네트워크에 관련한 시스템 구조들을 살펴보았습니다.

이제 네트워크 자체가 어떻게 동작하는지 좀더 세부적으로 살펴보도록 합시다.

![](1.png)

알필요는 없지만 가장 아래의 물리계층은 전기신호같은걸 다루는 계층이라고 합니다.

알필요는 없지만 상위 계층은 하위계층으로부터 구성되며,

이런 구조 덕분에 상위계층의 작업자는 하위계층까지 일일히 작업할 필요가 없어집니다.

- IP

> IP는 보내는 컴퓨터에서 받는 컴퓨터까지의 통신을 책임집니다.

> 호스트는 자기 자신의 주소를 로컬호스트라 불리는 127.0.0.1로 지정합니다.

> 호스트가 외부 망과 연결되면 공용 아이피 주소를 갖게 됩니다.

> 경찰이 아이피 추적을 한다면 이 공용 아이피를 추적하는 것입니다.

- TCP

> 통신할때 IP와 함께 사용되는 프로토콜입니다.

> IP가 데이터를 전송한다면 TCP는 그 데이터를 추적, 관리합니다.

우리가 상호작용하는 대부분의 통신은 TCP/IP기반입니다.